In [1]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt


In [2]:
def get_osm_data(place_name: str, tags: dict):
    """
    Pobiera dane z OpenStreetMap dla danego obszaru i tagów.

    Parameters:
        place_name (str): nazwa lokalizacji np. "Warszawa, Polska"
        tags (dict): słownik tagów OSM np. {"amenity": "parcel_locker"}

    Returns:
        geopandas.GeoDataFrame
    """
    print(f"Pobieranie danych OSM dla: {tags} w lokalizacji: {place_name}")
    gdf = ox.features_from_place(place_name, tags)
    return gdf


In [3]:
place = "Warszawa, Polska"
tags = {"building": True}

gdf = ox.features_from_place(place, tags)

# Eksploracja dostępnych tagów
print("Dostępne kolumny (tagi):")
for col in gdf.columns:
    if gdf[col].notna().any():
        unique_vals = gdf[col].dropna().unique()
        print(f"{col}: {len(unique_vals)} unikalnych wartości")
        print(f"    Przykładowe: {unique_vals[:5]}\n")


Dostępne kolumny (tagi):
geometry: 184464 unikalnych wartości
    Przykładowe: <GeometryArray>
[<POINT (21.249 52.214)>, <POINT (20.941 52.333)>, <POINT (21.026 52.197)>,
 <POINT (21.028 52.241)>, <POINT (20.941 52.333)>]
Length: 5, dtype: geometry

access: 6 unikalnych wartości
    Przykładowe: ['yes' 'no' 'customers' 'private' 'permissive']

building: 145 unikalnych wartości
    Przykładowe: ['bunker' 'yes' 'wayside_shrine' 'public' 'ruins']

bunker_type: 5 unikalnych wartości
    Przykładowe: ['mg_nest' 'gun_emplacement' 'ringstand_58c' 'pillbox'
 'bunker_type=mg_nest']

historic: 24 unikalnych wartości
    Przykładowe: ['yes' 'wayside_shrine' 'bunker' 'castle' 'building']

military: 3 unikalnych wartości
    Przykładowe: ['bunker' 'office' 'yes']

name: 3640 unikalnych wartości
    Przykładowe: ['Regelbau 514' 'Urząd Pocztowy 139' 'Wgórę' 'Żabka'
 'Centrum Wspinaczkowe On Sight']

ruins: 4 unikalnych wartości
    Przykładowe: ['yes' 'bunker' 'industrial' '1']

addr:street: 4760 uni

In [6]:
import osmnx as ox

# 1. Zdefiniuj obszar i tagi
place = "Warszawa, Polska"
tags = {
    "amenity": ["parcel_locker"],
    "highway": ["bus_stop"]
}

# 2. Pobierz dane
gdf = ox.features_from_place(place, tags)

# 3. Sprawdź dostępne kolumny
print("Dostępne kolumny w gdf:\n", gdf.columns.to_list())

# 4. Podziel na podzbiory
parcel_lockers = gdf[gdf["amenity"] == "parcel_locker"].copy()
bus_stops      = gdf[gdf["highway"] == "bus_stop"].copy()

# 5. Wybierz zestaw kolumn, który faktycznie istnieje
cols_to_show = ["osmid", "name", "operator", "brand", "ref", "public_transport", "geometry"]
available_parcel_cols = [c for c in cols_to_show if c in parcel_lockers.columns]
available_bus_cols    = [c for c in cols_to_show if c in bus_stops.columns]

# 6. Pokaż kilka pierwszych wierszy
print("\n=== Paczkomaty ===")
print(parcel_lockers[available_parcel_cols].head())

print("\n=== Przystanki autobusowe ===")
print(bus_stops[available_bus_cols].head())


Dostępne kolumny w gdf:
 ['geometry', 'bench', 'bin', 'bus', 'highway', 'lit', 'name', 'network', 'network:wikidata', 'public_transport', 'ref', 'shelter', 'tactile_paving', 'url', 'wheelchair', 'network:wikipedia', 'departures_board', 'route_ref', 'layer', 'location', 'name:network:pks_grojec', 'name:network:wtp', 'ref:wtp', 'url:wtp', 'check_date:bench', 'old_name', 'name:network:pks_polonus', 'url:pks_grojec', 'url:pks_polonus', 'name:de', 'mapillary', 'wikidata', 'towards', 'survey:date', 'old_ref', 'check_date:shelter', 'start_date', 'check_date', 'check_date:lit', 'operator', 'covered', 'image', 'local_ref', 'passenger_information_display', 'tram', 'kerb', 'note', 'network:short', 'url:bkm_otwock', 'url:opgp', 'source', 'disused:highway', 'name:signed', 'request_stop', 'operator:type', 'todo', 'disused:public_transport', 'wikimedia_commons', 'check_date:bin', 'check_date:tactile_paving', 'amenity', 'brand', 'brand:wikidata', 'operator:wikidata', 'operator:wikipedia', 'parcel_mail

In [7]:
parcel_lockers["brand_filled"] = parcel_lockers["brand"].fillna("Unknown")

# 2. Policz liczbę paczkomatów w podziale na brand
brand_counts = (
    parcel_lockers["brand_filled"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "brand", "brand_filled": "count"})
)

print(brand_counts)

                     count  count
0         Paczkomat InPost    972
1          Allegro One Box    205
2       DPD Pickup Station    116
3             Orlen Paczka     94
4                      DPD     52
5                  Allegro     49
6              DHL Pop Box     48
7                  Unknown     29
8          Appkomat InPost     27
9                   InPost     26
10                   Orlen     21
11                     DHL     20
12              AliExpress     15
13       Lodówkomat InPost      9
14                 Pocztex      7
15            DHL BOX 24/7      4
16           Poczta Polska      3
17              DPD Pickup      2
18    Pralniomat Q2 Pranie      2
19                 Cainiao      2
20                 SwipBox      2
21                   mebox      1
22                 arif.pl      1
23               Paczkomat      1
24     DPD Ppickup Station      1
25             DHL POP BOX      1
26          Allegro OneBox      1
27        DPD Pickup Point      1
28     Automat

In [8]:
# Załóżmy, że masz już GeoDataFrame `parcel_lockers`
# z kolumną 'brand' zawierającą surowe nazwy.

def categorize_brand(b):
    """Przyporządkuj nazwę do jednej z głównych marek lub 'Inne'."""
    if not isinstance(b, str):
        return "Inne"
    s = b.lower().replace('.', '').replace(',', '').strip()
    if "inpost" in s:
        return "InPost"
    if "dhl" in s:
        return "DHL"
    if "allegro" in s:
        return "Allegro"
    if "dpd" in s:
        return "DPD"
    if "orlen" in s:
        return "Orlen"
    if "aliexpress" in s:
        return "Aliexpress"
    if "poczta" in s and "polska" in s:
        return "Poczta Polska"
    return "Inne"

# 1) Tworzymy nową kolumnę z kategorią
parcel_lockers['brand_category'] = (
    parcel_lockers['brand']
    .fillna('')            # NaN → pusty string
    .apply(categorize_brand)
)

# 2) Grupujemy i zliczamy
brand_counts = (
    parcel_lockers['brand_category']
    .value_counts()
    .reset_index()
    .rename(columns={'index': 'brand', 'brand_category': 'count'})
)

print(brand_counts)


           count  count
0         InPost   1034
1        Allegro    255
2            DPD    172
3          Orlen    115
4            DHL     75
5           Inne     47
6     Aliexpress     15
7  Poczta Polska      3


In [9]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

# Funkcja kategoryzacji marki
def categorize_brand(b):
    if not isinstance(b, str):
        return "Inne"
    s = b.lower().replace('.', '').replace(',', '').strip()
    if "inpost" in s:
        return "InPost"
    if "dhl" in s:
        return "DHL"
    if "allegro" in s:
        return "Allegro"
    if "dpd" in s:
        return "DPD"
    if "orlen" in s:
        return "Orlen"
    if "aliexpress" in s:
        return "Aliexpress"
    if "poczta" in s and "polska" in s:
        return "Poczta Polska"
    return "Inne"

# Lista miast
cities = [
    "Warszawa, Polska", "Wrocław, Polska", "Łódź, Polska",
    "Poznań, Polska", "Białystok, Polska", "Gdańsk, Polska",
    "Bydgoszcz, Polska"
]

all_lockers = []
for city in cities:
    # 1) Pobranie paczkomatów w danym mieście
    tags = {"amenity": ["parcel_locker"]}
    gdf = ox.features_from_place(city, tags)
    lockers = gdf[gdf["amenity"] == "parcel_locker"].copy()
    
    # 2) Kategoryzacja brandu
    lockers["brand_category"] = lockers["brand"].fillna("").apply(categorize_brand)
    
    # 3) Dodanie kolumny 'city'
    lockers["city"] = city.split(",")[0]
    
    # 4) Wybór tylko potrzebnych kolumn
    all_lockers.append(lockers[["brand_category", "city", "geometry"]])

# 5) Połączenie w jeden GeoDataFrame
result_gdf = gpd.GeoDataFrame(
    pd.concat(all_lockers, ignore_index=True),
    crs="EPSG:4326"
)

# Rzut oka na wynik
print(result_gdf.head())
print("Łącznie paczkomatów pobranych:", len(result_gdf))


  brand_category      city                   geometry
0         InPost  Warszawa  POINT (21.04238 52.17507)
1         InPost  Warszawa   POINT (21.0911 52.22096)
2         InPost  Warszawa  POINT (21.09522 52.23292)
3         InPost  Warszawa   POINT (21.0329 52.17308)
4         InPost  Warszawa  POINT (20.93947 52.23772)
Łącznie paczkomatów pobranych: 5221


In [10]:
# 1) Stwórz kolumny lat/lon z geometrii
result_gdf["lon"] = result_gdf.geometry.x
result_gdf["lat"] = result_gdf.geometry.y

# 2) Wybierz kolumny do zapisu (bez obiektu geometry)
df_to_csv = result_gdf.drop(columns="geometry")

# 3) Zapisz do CSV
df_to_csv.to_csv("paczkomaty_by_brand_city.csv", index=False)

print("Zapisano plik paczkomaty_by_brand_city.csv, liczba wierszy:", len(df_to_csv))


ValueError: x attribute access only provided for Point geometries

In [ ]:


# Przykład: przystanki autobusowe
tags_bus_stops = {"highway": "bus_stop"}
bus_stops = get_osm_data("Warszawa, Polska", tags_bus_stops)

# Przykład: sklepy convenience (Żabka i podobne)
tags_convenience = {"shop": "convenience"}
convenience_stores = get_osm_data("Warszawa, Polska", tags_convenience)

# Przykład: parkingi
tags_parking = {"amenity": "parking"}
parkings = get_osm_data("Warszawa, Polska", tags_parking)

# Wyświetlenie liczby wyników
print("Liczba paczkomatów:", len(parcel_lockers))
print("Liczba przystanków:", len(bus_stops))
print("Liczba sklepów convenience:", len(convenience_stores))
print("Liczba parkingów:", len(parkings))

# 🗺️ Przykładowa wizualizacja
fig, ax = plt.subplots(figsize=(10, 10))
parcel_lockers.plot(ax=ax, color='red', markersize=10, label='Paczkomaty')
bus_stops.plot(ax=ax, color='blue', markersize=5, label='Przystanki')
convenience_stores.plot(ax=ax, color='green', markersize=5, label='Sklepy convenience')
parkings.plot(ax=ax, color='orange', markersize=5, label='Parking')

plt.legend()
plt.title("Wybrane obiekty OSM w Warszawie")
plt.show()
